# House Prices - Advanced Regression Techniques

* Author: [John Adeojo](https://www.john-adeojo.com/)
* Blog: [more projects on my medium blog](https://medium.com/@johnadeojo)
* LinkedIn: [Follow me](https://www.linkedin.com/in/john-adeojo/)

# Import Data 

In [1]:
import pandas as pd
import numpy as np

In [32]:
df_train = pd.read_csv(r"https://raw.githubusercontent.com/john-adeojo/kaggle_advanced_regression/main/data/01_raw/train%20(3).csv")
df_test = pd.read_csv(r"https://raw.githubusercontent.com/john-adeojo/kaggle_advanced_regression/main/data/01_raw/test%20(2).csv")

In [3]:
# do some quick data profiling with ydata profiling
# from ydata_profiling import ProfileReport

# profile = ProfileReport(df_train, title="Pandas Profiling Report: House Price Data")
# profile.to_file(r"C:\Users\johna\anaconda3\envs\kaggle-env\kaggle_advanced_regression\data\02_reports\testprofile_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# profile = ProfileReport(df_test, title="Pandas Profiling Report: House Price Data (Test)")
# profile.to_file(r"C:\Users\johna\anaconda3\envs\kaggle-env\kaggle_advanced_regression\data\02_reports\testprofile_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Data Wrangling & Cleaning

In [68]:
# define funtion te rplace missing vairbales

import pandas as pd

def impute_missing(df, test=False):
    
    if test == False:
        id_df = df['Id']
        y = df['SalePrice']
        df = df.drop(columns=['Id', 'SalePrice'])
    else:
        id_df = df['Id']
        df = df.drop(columns=['Id'])
    
    
    df['LotFrontage'] = df['LotFrontage'].fillna(0)
    df['Alley'] = df['Alley'].fillna('No Alley')
    df['MasVnrType'] = df['MasVnrType'].fillna('None')
    df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
    df['BsmtQual'] = df['BsmtQual'].fillna('None')
    df['BsmtCond'] = df['BsmtCond'].fillna('None')
    df['BsmtExposure'] = df['BsmtExposure'].fillna('None')
    df['BsmtFinType1'] = df['BsmtFinType1'].fillna('None')
    df['BsmtFinType2'] = df['BsmtFinType2'].fillna('None')
    df['Electrical'] = df['Electrical'].fillna('SBrkr')
    df['FireplaceQu'] = df['FireplaceQu'].fillna('None')
    df['GarageType'] = df['GarageType'].fillna('None')
    df['GarageYrBlt'] = df['GarageYrBlt'].fillna(9999)
    df['GarageFinish'] = df['GarageFinish'].fillna('None')
    df['GarageQual'] = df['GarageQual'].fillna('None')
    df['GarageCond'] = df['GarageCond'].fillna('None')
    df['PoolQC'] = df['PoolQC'].fillna('None')
    df['Fence'] = df['Fence'].fillna('None')
    df['MiscFeature'] = df['MiscFeature'].fillna('None')
    
    # missing in test 
    df['MSZoning'] = df['MSZoning'].fillna('RL')
    df['Utilities'] = df['Utilities'].fillna('AllPub')
    df['Exterior1st'] = df['Exterior1st'].fillna('VinylSd')  # replace with mode for area.
    df['Exterior2nd'] = df['Exterior2nd'].fillna('VinylSd')  # replace with mode for area.
    df['BsmtFinSF1'] = df['BsmtFinSF1'].fillna(0) # effectively no basement
    df['BsmtFinSF2'] = df['BsmtFinSF2'].fillna(0)
    df['BsmtUnfSF'] = df['BsmtUnfSF'].fillna(0)
    df['BsmtUnfSF'] = df['BsmtUnfSF'].fillna(0)
    df['TotalBsmtSF'] = df['TotalBsmtSF'].fillna(0)
    df['BsmtFullBath'] = df['BsmtFullBath'].fillna(0)
    df['BsmtHalfBath'] = df['BsmtHalfBath'].fillna(0)
    df['KitchenQual'] = df['KitchenQual'].fillna('TA')
    df['Functional'] = df['Functional'].fillna('Typ')
    df['Functional'] = df['Functional'].fillna('Typ')
    df['GarageCars'] = df['GarageCars'].fillna(0)
    df['GarageArea'] = df['GarageArea'].fillna(0) # if
    df['SaleType'] = df['SaleType'].fillna('WD') # if
    
    # convert int to object
    df['YearBuilt'] = df['YearBuilt'].astype('object')
    df['YearRemodAdd'] = df['YearRemodAdd'].astype('object')
    df['GarageYrBlt'] = df['GarageYrBlt'].astype('object')
    df['YrSold'] = df['YrSold'].astype('object')


    
    if test == False:
        return df, y, id_df
    else:
        return df, id_df



In [69]:
X, y, train_id = impute_missing(df_train, test=False)
X_test, test_id = impute_missing(df_test, test=True)

In [71]:
# Identify categorical and numeric columns
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_features = X.select_dtypes(include=['number']).columns.tolist()

In [85]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from umap import UMAP
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor


# construct the pipeline for data transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
        ('num', StandardScaler(), num_features)
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('umap', UMAP()),
    (XGBRegressor(objective='reg:squarederror'))
])


In [77]:
# Helper function
def generate_uniform_distribution(min_val, max_val, size=1, seed=19):
    # create a uniform distribution object
    dist = uniform(loc=min_val, scale=max_val-min_val)
    return dist


In [86]:
# split data 
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.25, random_state=42)

In [87]:
from scipy.stats import uniform, randint

param_grid = {
    'umap__n_components': randint(low=5, high=51),  # Search n_components between 5 and 50 with a step of 5
    'classifier__reg_lambda': generate_uniform_distribution(min_val=100, max_val=2000),  # Search for different regularization strengths
}

In [90]:
from sklearn.model_selection import RandomizedSearchCV
model_random_search = RandomizedSearchCV(pipeline, param_grid, cv=5, scoring='neg_root_mean_squared_error')
model_random_search.fit(X_train, y_train)

TypeError: 'XGBRegressor' object is not subscriptable

In [74]:
# run the pipeline

# fit pipeline on train
model_random_search.fit(X_train, y_train)

# # transform on train, test and validation
# X_train_trasnformed = pipeline.transform(X_train)
# X_validation_trasnformed = pipeline.transform(X_validation)
# X_test_trasnformed = pipeline.transform(X_test)

array([[ 2.1755495,  2.6248782],
       [ 2.2010977, -2.0198097],
       [ 6.930257 , -2.0688279],
       ...,
       [ 5.822583 , -3.7346938],
       [ 6.842134 , -0.9443781],
       [ 2.363676 ,  2.765911 ]], dtype=float32)

In [84]:
y.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64